In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

# 读取、缺失值填充

In [2]:
rating=pd.read_excel('C:/Users/78569/Desktop/电影推荐系统/data/toy.xlsx',index_col=0)
rating=rating.fillna(0)
rating

,action1,action2,action3,romantic1,romantic2,romantic3
user1,4.0,5.0,3.0,0.0,2.0,1.0
user2,5.0,3.0,3.0,2.0,2.0,0.0
user3,1.0,0.0,0.0,4.0,5.0,4.0
user4,0.0,2.0,1.0,4.0,0.0,3.0
user5,1.0,0.0,2.0,3.0,3.0,4.0


# 标准化

In [3]:
def standardize(x):
    new_x = (x-x.mean())/(x.max()-x.min())
    return new_x

rating_std = rating.apply(standardize)
rating_std

,action1,action2,action3,romantic1,romantic2,romantic3
user1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user5,-0.24,-0.4,0.066667,0.10,0.12,0.40


# i2i的相似度

In [4]:
item_similarity = cosine_similarity(rating_std.T)
print(item_similarity)

[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [5]:
item_similarity_df=pd.DataFrame(data=item_similarity,index=rating.columns,columns=rating.columns)
item_similarity_df

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


# 给一个电影做推荐

调用item_similarity_df，通过电影名和该用户给电影的评分来获取相似电影排名

In [6]:
def get_similar_movies(movie_name,user_rating):
    similar_movies = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_movies = similar_movies.sort_values(ascending=False)
    return similar_movies


In [7]:
get_similar_movies('action1',5)

action1      2.500000
action3      2.034204
action2      1.766722
romantic2   -0.063480
romantic1   -1.998527
romantic3   -2.285265
Name: action1, dtype: float64

In [8]:
get_similar_movies('romantic3',1)

action1      1.371159
action2      1.265061
action3      1.203271
romantic2   -0.590909
romantic1   -1.085620
romantic3   -1.500000
Name: romantic3, dtype: float64

# 给一个用户做推荐（该用户给多个电影做了评分）

创建一个用户

In [14]:
action_lover=[('action1',5),('romantic2',1),('romantic3',1)]

创建一个df来收集所有可以推荐给该用户的电影

In [15]:
similar_movies=pd.DataFrame()

In [16]:
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating))

In [17]:
similar_movies

,action1,action2,action3,romantic1,romantic2,romantic3
action1,2.500000,1.766722,2.034204,-1.998527,-0.063480,-2.285265
romantic2,0.038088,0.778499,0.569970,-0.222059,-1.500000,-0.590909
romantic3,1.371159,1.265061,1.203271,-1.085620,-0.590909,-1.500000


In [18]:
similar_movies.sum().sort_values(ascending=False)

action1      3.909247
action2      3.810282
action3      3.807445
romantic2   -2.154389
romantic1   -3.306206
romantic3   -4.376174
dtype: float64